In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import math
import pandas as pd
import os
from os import path
from tqdm import tqdm
import json
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime
import time
import random
from PIL import Image
import pickle
import joblib
import re

from sklearn import preprocessing
import tensorflow as tf
from keras.layers import Input,Dense,LSTM,Flatten,Dropout,concatenate,Conv1D,MaxPooling2D,Activation
from keras.layers import BatchNormalization
from keras.layers import Embedding
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# import keras_tuner
from tensorflow import keras
import tensorflow_hub as hub
import imgaug.augmenters as iaa
from tensorflow.keras.preprocessing import image, text, sequence
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

import warnings
warnings.filterwarnings("ignore")

## 1. Error Analysis

In [ ]:
#  Reaearch Paper Link: https://arxiv.org/pdf/1505.00468.pdf

def accuracy_metric(X,Y,encoded_features,model):
  
  predicted_Y = model.predict(encoded_features,verbose=0)
  predicted_class = tf.argmax(predicted_Y, axis=1, output_type=tf.int32)
  predicted_ans = labelencoder.inverse_transform(predicted_class)
  
  acc_val_lst = []
  for i in tqdm(range(len(Y))):
    acc_val = 0.0
    temp = 0
  
    for actual_ans in (list(X['answers'])[i]).split(","):
      if actual_ans == predicted_ans[i]:
        temp += 1
      
    if temp >= 3:
      acc_val = 1
    else:
      acc_val = float(temp)/3
  
    acc_val_lst.append(acc_val)
  
  return (sum(acc_val_lst)/len(Y))*100, acc_val_lst

In [ ]:
colab_path = "/content/drive/MyDrive/Applied AI/"
test_image= pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/test_image_50k_0711.pkl', 'rb'))
tokenizer_50k = pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/tokenizer_50k.pkl', 'rb'))
model = tf.keras.models.load_model('/content/drive/MyDrive/Applied AI/CS2/model/model_2lstm_vgg19_50k_1011_50.h5')
labelencoder = pickle.load(open('/content/drive/MyDrive/Applied AI/CS2/model/labelencoder.pkl', 'rb'))

test_sequences = tokenizer_50k.texts_to_sequences(list(X_test['question_preprocessed']))
test_que = pad_sequences(test_sequences, maxlen=22, padding='post')
print(test_image.shape, test_que.shape)

(4501, 224, 224, 3) (4501, 22)


In [ ]:
Test_accuracy, accuracy_lst = accuracy_metric(X_test,Y_test,[test_que,test_image],model)
print("\nTest Accuracy:",round(Test_accuracy,3))

100%|██████████| 4501/4501 [00:01<00:00, 4074.09it/s]


Test Accuracy: 44.242


In [ ]:
test_df = X_test
test_df['answer'] = list(labelencoder.inverse_transform(list(y_test)))
test_df['class_label'] = list(y_test)
test_df['Accuracy'] = list(accuracy_lst)
test_df.head()

,image_id,question_preprocessed,answers,answer,class_label,Accuracy
21712,train2014/COCO_train2014_000000060378.jpg,what color is the bike,"black,black,gray,black,black,blue,black,red,bl...",black,127,0.000000
11079,train2014/COCO_train2014_000000051555.jpg,whose apt is this,"man,man,not possible,man on lefts,no clue,man ...",man,528,0.666667
4295,train2014/COCO_train2014_000000029523.jpg,is she wearing a bathing suit,"no,no,no,no,no,no,no,no,no,no",no,583,0.000000
30709,train2014/COCO_train2014_000000440344.jpg,how many bicycles are there,"2,2,2,2,2,2,3,2,2,2",2,14,0.000000
28512,train2014/COCO_train2014_000000360236.jpg,is this person standing,"yes,yes,yes,yes,yes,yes,yes,yes,yes,yes",yes,994,0.000000


In [ ]:
set(test_df['Accuracy'])

{0.0, 0.3333333333333333, 0.6666666666666666, 1.0}

In [ ]:
print(f"Only {len(test_df[(test_df.Accuracy != 0) & (test_df.Accuracy != 1)])} datapoints have accuracy 0.33 or 0.66, so add them with correct datapoints.")

Only 458 datapoints have accuracy 0.33 or 0.66, so add them with correct datapoints.


In [ ]:
correct_df = test_df[test_df.Accuracy != 0]
incorrect_df = test_df[test_df.Accuracy == 0]
print("Number of Datapoints in correct_df:",len(correct_df))
print("Number of Datapoints in incorrect_df:",len(incorrect_df))

Number of Datapoints in correct_df: 2247
Number of Datapoints in incorrect_df: 2254


In [ ]:
def top_20_firstword_que(data_f):
  first_word = pd.Series(list(data_f['question_preprocessed'])).apply(lambda x: x.split()[0])
  data_f["que_word"] = list(first_word)
  df = pd.DataFrame((data_f.groupby(['que_word']).count()).index,columns=["que_word"])
  df["que_count"] = list(data_f.groupby(['que_word']).count()['question_preprocessed'])
  df["que%"] = round(df["que_count"]/len(correct_df)*100,2)
  df = df.sort_values(by='que_count',ascending=False)
  return [tuple(x) for x in df.values[:20]]

In [ ]:
def top_20_ans(data_f):
  df = pd.DataFrame((data_f.groupby(['answer']).count()).index,columns=["answer"])
  df["answer_count"] = list(data_f.groupby(['answer']).count()['answers'])
  df["answer%"] = round(df["answer_count"]/len(correct_df)*100,2)
  df = df.sort_values(by='answer_count',ascending=False)
  return [tuple(x) for x in df.values[:20]]

### 1.1 Correct Predicted Datapoints Analysis

In [ ]:
top_20_firstword_que(correct_df)

[('is', 935, 41.61),
 ('what', 489, 21.76),
 ('are', 259, 11.53),
 ('how', 243, 10.81),
 ('does', 101, 4.49),
 ('do', 38, 1.69),
 ('can', 28, 1.25),
 ('which', 27, 1.2),
 ('could', 21, 0.93),
 ('where', 15, 0.67),
 ('has', 14, 0.62),
 ('did', 13, 0.58),
 ('would', 11, 0.49),
 ('will', 10, 0.45),
 ('who', 7, 0.31),
 ('was', 6, 0.27),
 ('if', 4, 0.18),
 ('overcast', 3, 0.13),
 ('yes', 3, 0.13),
 ('should', 2, 0.09)]

In [ ]:
top_20_ans(correct_df)

[('no', 735, 32.71),
 ('yes', 693, 30.84),
 ('2', 102, 4.54),
 ('1', 85, 3.78),
 ('white', 81, 3.6),
 ('3', 24, 1.07),
 ('brown', 17, 0.76),
 ('right', 14, 0.62),
 ('tennis', 13, 0.58),
 ('blue', 13, 0.58),
 ('gray', 13, 0.58),
 ('red', 13, 0.58),
 ('bathroom', 13, 0.58),
 ('left', 12, 0.53),
 ('wood', 11, 0.49),
 ('black', 9, 0.4),
 ('4', 9, 0.4),
 ('0', 8, 0.36),
 ('pizza', 8, 0.36),
 ('skateboarding', 7, 0.31)]

### 1.2 Incorrect Predicted Datapoints Analysis

In [ ]:
top_20_firstword_que(incorrect_df)

[('what', 1151, 51.22),
 ('is', 387, 17.22),
 ('how', 354, 15.75),
 ('are', 109, 4.85),
 ('where', 62, 2.76),
 ('does', 39, 1.74),
 ('which', 26, 1.16),
 ('why', 22, 0.98),
 ('do', 19, 0.85),
 ('who', 14, 0.62),
 ('can', 13, 0.58),
 ('whats', 12, 0.53),
 ('has', 8, 0.36),
 ('could', 5, 0.22),
 ('would', 4, 0.18),
 ('the', 4, 0.18),
 ('of', 3, 0.13),
 ('was', 3, 0.13),
 ('if', 2, 0.09),
 ('in', 2, 0.09)]

In [ ]:
top_20_ans(incorrect_df)

[('yes', 312, 13.89),
 ('no', 241, 10.73),
 ('1', 63, 2.8),
 ('3', 53, 2.36),
 ('blue', 52, 2.31),
 ('0', 51, 2.27),
 ('black', 51, 2.27),
 ('red', 49, 2.18),
 ('2', 43, 1.91),
 ('4', 40, 1.78),
 ('green', 38, 1.69),
 ('yellow', 29, 1.29),
 ('brown', 28, 1.25),
 ('5', 26, 1.16),
 ('white', 25, 1.11),
 ('nothing', 18, 0.8),
 ('none', 15, 0.67),
 ('frisbee', 15, 0.67),
 ('pink', 13, 0.58),
 ('baseball', 13, 0.58)]

### 1.3 Observations

* In Correctly predicted datapoints, most questions start with the 'is' word and in Incorrectly predicted datapoints, most questions start with the 'what' word.
* In Correctly predicted datapoints, approx 60% of questions have 'yes' or 'no' answers.
* In Incorrectly predicted datapoints, only 20% of questions have 'yes' or 'no' answers and most questions have numeric, color, and other types of answers.
* From this Error analysis, we can conclude that questions that have 'yes' or 'no' answer types are mostly correctly predicted and questions that have numeric or other answer types are a little bit difficult to predict.